In [3]:
from utils.read import readExcel

import pandas as pd

In [4]:
df = readExcel(
    "./../../upgrade_mandi/raw-sheets-dump/Upgrade 27 september 2025.xlsx",
    sheetName="Sheet1",
)

df.columns = [column.strip().lower() for column in df.columns]

df["indents"] = df["indents"].apply(int)

In [5]:
df

,date,city,store id,sku_id,store_name,vendor,item_code,product_name,weight,indents,supplier id,cost,mrp,po number,entity name
0,2025-09-27,Nagpur,1392084.0,OKAF6BM41C,Byramji,Upgrade Mandi,5796.0,Banana Flower,1piece,1,74227878.0,40.0,50.00,20250927-1392084-74227878,RAJIDI RETAIL PVT LTD
1,2025-09-27,Nagpur,1397035.0,IQQH3Y3FCI,Nandavan,Upgrade Mandi,5796.0,Banana Flower,1piece,1,74227878.0,40.0,50.00,20250927-1397035-74227878,SWINSTA ENT PRIVATE LIMITED
2,2025-09-27,Nagpur,1397624.0,FCRFZPI8CZ,Dharampeth,Upgrade Mandi,5796.0,Banana Flower,1piece,1,74227878.0,40.0,50.00,20250927-1397624-74227878,SWINSTA ENT PRIVATE LIMITED
3,2025-09-27,Nagpur,1403419.0,J08LRYN8AU,Ayodhaya Nagar,Upgrade Mandi,5796.0,Banana Flower,1piece,1,74227878.0,40.0,50.00,20250927-1403419-74227878,RAJIDI RETAIL PVT LTD
4,2025-09-27,Nagpur,1392084.0,U7T1YFDT58,Byramji,Upgrade Mandi,11054.0,Bel Patta,1pack,1,74227878.0,15.0,18.75,20250927-1392084-74227878,RAJIDI RETAIL PVT LTD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2025-09-27,Nagpur,1392532.0,SXM30PLTJ5,Manish Nagar,Upgrade Mandi,9497.0,Yellow Shevanti Flowers,100g,2,74227878.0,45.0,56.25,20250927-1392532-74227878,RAJIDI RETAIL PVT LTD
139,2025-09-27,Nagpur,1393571.0,F04ZK4FSSP,Mahal,Upgrade Mandi,9497.0,Yellow Shevanti Flowers,100g,1,74227878.0,45.0,56.25,20250927-1393571-74227878,RAJIDI RETAIL PVT LTD
140,2025-09-27,Nagpur,1397035.0,PF24KCFRTN,Nandavan,Upgrade Mandi,9497.0,Yellow Shevanti Flowers,100g,1,74227878.0,45.0,56.25,20250927-1397035-74227878,SWINSTA ENT PRIVATE LIMITED
141,2025-09-27,Nagpur,1397624.0,PRJGKMQSVF,Dharampeth,Upgrade Mandi,9497.0,Yellow Shevanti Flowers,100g,1,74227878.0,45.0,56.25,20250927-1397624-74227878,SWINSTA ENT PRIVATE LIMITED


In [6]:
df.columns

Index(['date', 'city', 'store id', 'sku_id', 'store_name', 'vendor',
       'item_code', 'product_name', 'weight', 'indents', 'supplier id', 'cost',
       'mrp', 'po number', 'entity name'],
      dtype='object')

In [7]:
groupByProductName = df.groupby("item_code")

In [8]:
final = pd.DataFrame(
    {
        "product_name": groupByProductName["product_name"]
        .unique()
        .apply(lambda names: names[0]),
        "weight": groupByProductName["weight"]
        .unique()
        .apply(lambda weights: weights[0]),
        "indents": groupByProductName["indents"].sum().astype(int),
    }
)
final.reset_index(inplace=True, drop=True)
final.columns = ["Product_Name", "Weight", "Indents"]

In [9]:
final["Sr No"] = range(1, len(final) + 1)
final["Final"] = (
    final["Weight"]
    .apply(
        lambda weight: "".join(
            [char for char in weight if 48 <= ord(char) and ord(char) <= 57]
        )
    )
    .astype(int)
    * final["Indents"]
)
final = pd.concat(
    [
        final,
        pd.DataFrame(
            {
                "Sr No": [""],
                "Product_Name": [""],
                "Weight": [""],
                "Indents": [f'Total: {final["Indents"].sum().astype(int)}'],
                "Final": [""],
            }
        ),
    ]
)

In [10]:
final[["Sr No", "Product_Name", "Weight", "Indents", "Final"]].to_excel(
    "./../Final.xlsx",
    sheet_name="Final Report",
    index=False,
    engine="openpyxl",
    freeze_panes=(1, 0),
    columns=["Sr No", "Product_Name", "Weight", "Indents", "Final"],
)